In [ ]:
# default_exp apigateway

# apigateway schema for lambda proxy integration


In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from dataclasses import field
from dataclasses import dataclass, field
from dataclasses_json import dataclass_json, Undefined
from pprint import pformat
from typing import Optional, List, Callable, Any
import ujson as json

@dataclass_json
@dataclass
class Response:
  '''
    parse response from apigateway
  '''
  body: Optional[str]
  statusCode: int = 200
  headers: dict = field(default_factory = dict)
  @classmethod
  def parseBody(cls, dictInput:dict):
    response = cls.fromDict(dictInput)
    return response.body
  @classmethod
  def parseHeaders(cls, dictInput:dict):
    response = cls.fromDict(dictInput)
    return response.headers
  
  @classmethod
  def fromDict(cls, dictInput:dict):
    '''
      output object from Dict,
      dictInput should follow apigateway proxy integration
    '''
    body = dictInput.pop('body')
    return cls(
      body = json.loads(body),
      **dictInput
    )
  @classmethod
  def getReturn(cls, body:dict, headers:dict = {
            'Access-Control-Allow-Headers': '*',
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Methods': '*',
        }, 
        statusCode:int = 200)->dict:
    '''
      output dictionary which is suitable for apigateway proxy integration return
    '''
    returnObj = cls(
      body = json.dumps(body),
      headers = headers,
      statusCode = statusCode
                   ).to_dict()
    return returnObj
  @classmethod
  def returnError(cls, message:str, statusCode:int = 400, **kwargs)->dict:
    '''add override statusCode Here by putting in the values directly'''
    return cls.getReturn(statusCode = statusCode, body = {'error': message})
  @classmethod
  def returnSuccess(cls, body:dict = {}, statusCode:int = 200, **kwargs)->dict:
    '''add override statusCode Here'''
    return cls.getReturn(statusCode = statusCode, body = body, **kwargs)
  
@dataclass_json(undefined=Undefined.EXCLUDE)
@dataclass
class Event:
  '''
    parse event from apigateway
  '''
  body: str
    
  httpMethod: Optional[str] = None
  multiValueHeaders: Optional[dict] = None
  multiValueQueryStringParameters: Optional[dict] = None
  path: Optional[str] = None
  pathParameters: Optional[dict] = None
  queryStringParameters: Optional[dict] = None
  requestContext: Optional[dict] = None
    
  headers: dict = field(default_factory = dict)
  statusCode: int = 200
  isBase64Encoded: bool = False
    
  def getBody(self,*args):
    try:
      return json.loads(self.body)
    except:
      return Event.parseBody(self,*args)
  def getProducts(self):
    return Products.from_json(self.body)
  def getKey(self, key='product'):
    return self.body.get(key)
  
  key = lambda self: json.loads(self.body)['key']
  firstKey = lambda self: next(iter(json.loads(self.body).items()))
  @classmethod
  def parseBody(cls, event, *args):
    return cls.from_dict(event).getBody()
  @classmethod
  def parseHeaders(cls, event, *args):
    return cls.from_dict(event).headers
  @classmethod
  def parseQuery(cls, event, *args):
    return cls.from_dict(event).queryStringParameters
  @classmethod
  def parsePath(cls, event, *args):
    return cls.from_dict(event).path
  @classmethod
  def getInput(cls, body={},headers={},statusCode=200,queryStringParameters={}):
    '''
    crate innput for mocking up the object
    '''
    return cls(body=json.dumps(body),headers=headers,
               statusCode=statusCode,queryStringParameters=queryStringParameters).to_dict()
  @classmethod
  def parseDataClass(cls, customClass, event):
    body = cls.getBody(event)
    try:
      return customClass.from_dict(body)
    except Exception as e:
      raise Exception(f'unable to parse input{e}, should have the schema {customClass.__doc__},\
        but the current input is {body}')
  
@dataclass_json
@dataclass
class Product:
  cprcode: str
  iprcode: str
  oprcode: str
  ordertype: str
  pr_abb: str
@dataclass_json
@dataclass
class Products:
  products: List[Product]

# Examples

In [ ]:
event = Event(body = json.dumps({'test':'test'})).to_dict()
print(event)

{'body': '{"test":"test"}', 'httpMethod': None, 'multiValueHeaders': None, 'multiValueQueryStringParameters': None, 'path': None, 'pathParameters': None, 'queryStringParameters': None, 'requestContext': None, 'headers': {}, 'statusCode': 200, 'isBase64Encoded': False}


In [ ]:
Event.parseBody(event)

{'test': 'test'}

## Returning response

### success

In [ ]:
#### just body
Response.returnSuccess(body = {'success':'true'})

{'body': '{"success":"true"}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

In [ ]:
#### override statusCode
Response.returnSuccess(statusCode = 206, body = {'success':'true'})

{'body': '{"success":"true"}',
 'statusCode': 206,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

In [ ]:
#### without body
Response.returnSuccess(body = None)

{'body': 'null',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

In [ ]:
# standard
Response.returnSuccess(body = {})

{'body': '{}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

### Error

In [ ]:
Response.returnError(statusCode = 402, message='error')

{'body': '{"error":"error"}',
 'statusCode': 402,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

responseDict = Response.returnSuccess(body = {'success':'true'})
Response.fromDict(responseDict)

In [ ]:
event = Event.getInput(body={'input':'123'}, queryStringParameters={'test':'123'})
event

{'body': '{"input":"123"}',
 'httpMethod': None,
 'multiValueHeaders': None,
 'multiValueQueryStringParameters': None,
 'path': None,
 'pathParameters': None,
 'queryStringParameters': {'test': '123'},
 'requestContext': None,
 'headers': {},
 'statusCode': 200,
 'isBase64Encoded': False}

## Parsing requests

In [ ]:
with open('./testData/sampleEvent.json') as f:
  sampleEvent = json.load(f)['input']

### get inidividual features

#### body

In [ ]:
body = Event.parseBody(sampleEvent)
assert body == json.loads(Event.from_dict(sampleEvent).body)
print(body)

{'a': 1}


In [ ]:
#### header
headers = Event.parseHeaders(sampleEvent)
assert headers == Event.from_dict(sampleEvent).headers
print(headers)

{'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'cache-control': 'no-cache', 'CloudFront-Forwarded-Proto': 'https', 'CloudFront-Is-Desktop-Viewer': 'true', 'CloudFront-Is-Mobile-Viewer': 'false', 'CloudFront-Is-SmartTV-Viewer': 'false', 'CloudFront-Is-Tablet-Viewer': 'false', 'CloudFront-Viewer-Country': 'US', 'Content-Type': 'application/json', 'headerName': 'headerValue', 'Host': 'gy415nuibc.execute-api.us-east-1.amazonaws.com', 'Postman-Token': '9f583ef0-ed83-4a38-aef3-eb9ce3f7a57f', 'User-Agent': 'PostmanRuntime/2.4.5', 'Via': '1.1 d98420743a69852491bbdea73f7680bd.cloudfront.net (CloudFront)', 'X-Amz-Cf-Id': 'pn-PWIJc6thYnZm5P0NMgOUglL1DYtl0gdeJky8tqsg8iS_sgsKD1A==', 'X-Forwarded-For': '54.240.196.186, 54.182.214.83', 'X-Forwarded-Port': '443', 'X-Forwarded-Proto': 'https'}


In [ ]:
#### query String
query = Event.parseQuery(sampleEvent)
assert query ==Event.from_dict(sampleEvent).queryStringParameters
print(query)

{'name': 'me', 'multivalueName': 'me'}


In [ ]:
#### path
path = Event.parsePath(sampleEvent)
assert path == Event.from_dict(sampleEvent).path
print(path)

/hello/world


## Parsing data class directly

In [ ]:
productDict = Product(
  cprcode='123',
  iprcode='123',
  oprcode= '12343',
  ordertype='3225',
  pr_abb='4563'
).to_dict()
event = Event.getInput(productDict)
Event.parseDataClass(Product, event)

Product(cprcode='123', iprcode='123', oprcode='12343', ordertype='3225', pr_abb='4563')

In [ ]:
import re
from beartype import beartype
@beartype
def increaseVersion(string:str)->str:
  '''
  increase version number of the pip package by 1
  '''
  match = next(re.finditer(r'\d.\d.\d*', string))
  lastDigits = int(match.group().split('.')[-1])
  newSubVersionList = match.group().split('.')
  newSubVersionList[-1] = str(lastDigits + 1)
  newVersionString = '.'.join(newSubVersionList)
  wholeString = re.sub(r'\d.\d.\d*',newVersionString,string)
  return wholeString

@beartype
def loadFile(fileName:str= './settings.ini')->str:
  with open(fileName, 'r') as f:
    file = f.read()
    return file
  
@beartype
def saveFile(data,fileName='./settings.ini.bak')->str:
  with open(fileName, 'w') as f:
    f.write(data)
    return data
  
@beartype
def replaceContent(originalContent:str)->str:
  newFile = ''
  for line in file.split('\n'):
    newFile += '\n'
    if 'version' in line:
      newFile += increaseVersion(line)
    else:
      newFile += line

def testIncreaseVersion():
  fileName = 'settings.ini'
  tempFileName = 'settings.ini.bak'
  fileString:str = loadFile(fileName)
  print('testing load file')
  assert fileString == open(fileName, 'r').read()
  print('testing save file')
  saveFile(fileString, fileName = tempFileName)
  assert fileString == open(tempFileName, 'r').read()
  print('test increase version')

# !cat settings.ini.bak
testIncreaseVersion()
      
    

testing load file
testing save file
test increase version


In [ ]:
string = 'version = 0.0.14'
increaseVersion(string)

'version = 0.0.15'

# APIGATEWAY schemas

## Requests

In [ ]:
import yaml
schema = {
    "resource": "Resource path",
    "path": "Path parameter",
    "httpMethod": "Incoming request's method name",
    "headers": "{String containing incoming request headers}",
    "multiValueHeaders": "{List of strings containing incoming request headers}",
    "queryStringParameters": "{query string parameters }",
    "multiValueQueryStringParameters": "{List of query string parameters}",
    "pathParameters":  "{path parameters}",
    "stageVariables": "{Applicable stage variables}",
    "requestContext": "{Request context, including authorizer-returned key-value pairs}",
    "body": "A JSON string of the request payload.",
    "isBase64Encoded": "A boolean flag to indicate if the applicable request payload is Base64-encoded",
}
print(yaml.dump(schema))

body: A JSON string of the request payload.
headers: '{String containing incoming request headers}'
httpMethod: Incoming request's method name
isBase64Encoded: A boolean flag to indicate if the applicable request payload is Base64-encoded
multiValueHeaders: '{List of strings containing incoming request headers}'
multiValueQueryStringParameters: '{List of query string parameters}'
path: Path parameter
pathParameters: '{path parameters}'
queryStringParameters: '{query string parameters }'
requestContext: '{Request context, including authorizer-returned key-value pairs}'
resource: Resource path
stageVariables: '{Applicable stage variables}'



## Response

In [ ]:
schema = {
    "isBase64Encoded": 'bool',
    "statusCode": 'int',
    "headers": { "headerName": "headerValue" },
    "multiValueHeaders": { "headerName": ["headerValue", "headerValue2"]   },
    "body": "..."
}
print(yaml.dump(schema))

body: '...'
headers:
  headerName: headerValue
isBase64Encoded: bool
multiValueHeaders:
  headerName:
  - headerValue
  - headerValue2
statusCode: int

